# LSTM model
We also tried ARIMA and other types of NN, but:
 - ARIMA is insanely slow, so it is unlikely that it would fit in this contex of predicting prices, where we are counting on every second
 - Other types of NNs are probably viable if pre-trained, but LSTM showed best results

In [ ]:
da_price.join(predicted_da_price)

In [ ]:
prices = pd.DataFrame({'da_price': da_price, 
                       'predicted_da_price': predicted_da_price,
                       'wind_pred': wind_pred, 
                       'wind_actual': wind_actual, 
                       'solar_pred': solar_pred, 
                       'solar_actual': da_price, 
                       'combined_renewables': combined_renewables, 
                       'predicted_renewables': predicted_renewables})

In [ ]:
print(combined_renewables)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [ ]:
# Read data:
volue_data = pd.read_csv('../data/volue_data.csv')
volue_data = volue_data.rename(columns={'Unnamed: 0': 'ts'})

In [ ]:
volue_data['ts'] = pd.to_datetime(volue_data['ts'], utc = True)

In [ ]:
volue_data

In [ ]:
volue_data = volue_data[:191]

In [ ]:
# Read data:
series = TimeSeries.from_dataframe(volue_data, 'ts', ['da_price'])

# Create training and validation sets:
train, val = series.split_after(pd.Timestamp('2021-03-20 14:45:00+00:00'))

In [ ]:

# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

In [ ]:
my_model = RNNModel(
    model='LSTM',
    input_chunk_length=12,
    output_chunk_length=1,
    hidden_size=25,
    n_rnn_layers=1,
    dropout=0.4,
    batch_size=16,
    n_epochs=400,
    optimizer_kwargs={'lr': 1e-3}, 
    model_name='Air_RNN',
    log_tensorboard=True,
    random_state=42
)

In [ ]:
my_model.fit(train_transformed, val_series=val_transformed, verbose=True)  # 107

In [ ]:
def eval_model(model):
    pred_series = model.predict(n=26)
    plt.figure(figsize=(8,5))
    series_transformed.plot(label='actual')
    pred_series.plot(label='forecast')
    plt.title('MAPE: {:.2f}%'.format(mape(pred_series, val_transformed)))
    plt.legend();
    
eval_model(my_model)

In [ ]:
best_model = RNNModel.load_from_checkpoint(model_name='Air_RNN', best=True)
eval_model(best_model)

### Backtesting

In [ ]:
backtest_series = my_model.historical_forecasts(series_transformed,
                                                start=pd.Timestamp('2021-03-20 14:45:00+00:00'),
                                                forecast_horizon=6,
                                                retrain=False,
                                                verbose=True)

In [ ]:
plt.figure(figsize=(8,5))
series_transformed.plot(label='actual')
backtest_series.plot(label='backtest')
plt.legend()
print('MAPE: {:.2f}%'.format(mape(transformer.inverse_transform(series_transformed), 
                                  transformer.inverse_transform(backtest_series))))

In [ ]:
backtest_series.plot(label='LSTM')
plt.legend();